In [1]:
import spacy
from spacy import displacy
import spacy_annotator as spa
import pandas as pd
import numpy as np
import os

os.getcwd()

ModuleNotFoundError: No module named 'spacy_annotator'

In [2]:
! pip install spacy_annotator

  Using cached spacy_annotator-2.1.3-py3-none-any.whl (6.0 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
platformdirs 3.11.0 requires typing-extensions>=4.7.1; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
flake8 5.0.4 requires importlib-metadata<4.3,>=1.1.0; python_version < "3.8", but you have importlib-metadata 6.7.0 which is incompatible.
fake-useragent 1.2.1 requires importlib-metadata~=4.0; python_version < "3.8", but you have importlib-metadata 6.7.0 which is incompatible.


In [ ]:
#nlp = spacy.blank("en") # load a new spacy model for first time 
nlp = spacy.load("output/model-best") # load custom model after first time train

In [ ]:
# prepare train data
import spacy_annotator as spa

train_data=pd.DataFrame({
    "text":pd.read_parquet('./theEdgeDevSample.parquet').loc[91:100]['Content'].tolist()})

train_data=train_data[~((train_data['text'].str.contains('[\u4e00-\u9fff]+'))|(train_data['text'].isnull()))].reset_index(drop=True)
train_data['text']=train_data['text'].str.replace(r'(.+\d\)\:)|(.+[A-Z]\):)|(.+[A-Z]:)','',regex=True).str.strip()
train_data

### Reference for Text Annotation

> - <sup>PERSON:      People, including fictional.</sup>
> - <sup>NORP:        Nationalities or religious or political groups.</sup>
> - <sup>FAC:         Buildings, airports, highways, bridges, etc.</sup>
> - <sup>ORG:         Companies, agencies, institutions, etc.</sup>
> - <sup>GPE:         Countries, cities, states.</sup>
> - <sup>LOC:         Non-GPE locations, mountain ranges, bodies of water.</sup>
> - <sup>PRODUCT:     Objects, vehicles, foods, etc. (Not services.)</sup>
> - <sup>EVENT:       Named hurricanes, battles, wars, sports events, etc.</sup>
> - <sup>WORK_OF_ART: Titles of books, songs, etc.</sup>
> - <sup>LAW:         Named documents made into laws.</sup>
> - <sup>LANGUAGE:    Any named language.</sup>
> - <sup>DATE:        Absolute or relative dates or periods.</sup>
> - <sup>TIME:        Times smaller than a day.</sup>
> - <sup>PERCENT:     Percentage, including ”%“.</sup>
> - <sup>MONEY:       Monetary values, including unit.</sup>
> - <sup>QUANTITY:    Measurements, as of weight or distance.</sup>
> - <sup>ORDINAL:     “first”, “second”, etc.</sup>
> - <sup>CARDINAL:    Numerals that do not fall under another type.</sup>

In [ ]:
annotator=spa.Annotator(labels=["GPE","PERSON","ORG","FAC","MONEY","NORP","LOC","PRODUCT","EVENT","PERCENT","WORK_OF_ART","TIME","ORDINAL","CARDINAL","QUANTITY","LAW"],model=nlp) #annotate train data

In [ ]:
df_labels=annotator.annotate(df=train_data,col_text="text",shuffle=False)

In [ ]:
spacy_annotations = annotator.to_spacy(df_labels,"./train/train_1_100.spacy")

In [ ]:
! python3 -m spacy init fill-config base_config_gpu.cfg config.cfg #mac gpu
! python3 -m spacy train config.cfg --output ./output_new --paths.train ./train_new --paths.dev ./train_new #--gpu-id 0 mac